#### 自动创建finetune/data文件夹下的yaml文件

In [ ]:
import os

# 目标文件夹路径
output_dir = "/home/xiongshiwei/uni2ts/cli/conf/finetune/data/"  
os.makedirs(output_dir, exist_ok=True)

# 定义数据集
domain_dataset = {
    'Energy': ['CAISO_AL' 'NYISO_AL', 'Wind'],
    'Healthcare': ['Covid-19', 'illness'],
    'Environment': ['weather', 'AQShunyi', 'AQWan'],
    'Nature': ['CzeLan', 'ZafNoo'],
    'Traffic': ['M_DENSE', 'METR_LA', 'NYCTAXI_Inflow', 'NYCTAXI_Outflow'],
    'Finance': ['Exchange_rate', 'IBM_m', 'MSFT_w', 'NASDAQ_w'],
    'Web': ['Wike2000', 'BizITObs-Application', 'BizITObs-Service'],
    'CloudOps': ['BizITObs-L2C', 'Bitbrains-rnd100-cpuusage', 'Bitbrains-rnd100-memoryusage']
}

# YAML 文件内容模板
yaml_template = lambda name: f"""_target_: uni2ts.data.builder.simple.SimpleDatasetBuilder
dataset: {name}
weight: 1
"""

# 创建 YAML 文件
for domain, datasets in domain_dataset.items():
    for dataset in datasets:
        file_path = os.path.join(output_dir, f"{dataset}.yaml")
        with open(file_path, "w") as f:
            f.write(yaml_template(dataset))
        print(f"✅ Created: {file_path}")



✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/NYISO_AL.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/Wind.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/Covid-19.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/illness.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/weather.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/AQShunyi.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/AQWan.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/CzeLan.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/ZafNoo.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/M_DENSE.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/METR_LA.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/NYCTAXI_Inflow.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/data/NYCTAXI_Outflow.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune

#### 自动创建finetune/val_data文件夹下的yaml文件

In [ ]:
import os
import yaml

output_dir = "/home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/"
os.makedirs(output_dir, exist_ok=True)

Domain = ['Energy', 'Healthcare', 'Environment', 'Nature', 'Traffic', 'Finance', 'Web', 'CloudOps']

DomainDataset = {
    'Energy': ['CAISO_AL', 'NYISO_AL', 'Wind'],
    'Healthcare': ['Covid-19', 'illness'],
    'Environment': ['weather', 'AQShunyi', 'AQWan'],
    'Nature': ['CzeLan', 'ZafNoo'],
    'Traffic': ['M_DENSE', 'METR_LA', 'NYCTAXI_Inflow', 'NYCTAXI_Outflow'],
    'Finance': ['Exchange_rate', 'IBM_m', 'MSFT_w', 'NASDAQ_w'],
    'Web': ['Wike2000', 'BizITObs-Application', 'BizITObs-Service', 'BizITObs-L2C'],
    'CloudOps': ['Bitbrains-rnd100-cpuusage', 'Bitbrains-rnd100-memoryusage']
}

Seqlen = {
    'Energy': ['long', 'long', 'long'],
    'Healthcare': ['short', 'short'],
    'Environment': ['long', 'long', 'long'],
    'Nature': ['long', 'long'],
    'Traffic': ['long', 'long', 'short', 'short'],
    'Finance': ['long', 'short', 'short', 'short'],
    'Web': ['short', 'long', 'long', 'long'],
    'CloudOps': ['short', 'short']
}

offset = {
    'Energy': ['6148', '11974' ,'34071'],
    'Healthcare': ['974', '676'],
    'Environment': ['36887', '24545', '24545'],
    'Nature': ['13954', '13458'],
    'Traffic': ['12264', '23990', '1529', '1529'],
    'Finance': ['5312', '531', '1424', '541'],
    'Web': ['554', '6184', '6185', '22378'],
    'CloudOps': ['1306', '1306']
}

eval_length = {
    'Energy': ['878', '1711' ,'4867'],
    'Healthcare': ['139', '97'],
    'Environment': ['5270', '3506', '3506'],
    'Nature': ['1993', '1923'],
    'Traffic': ['1752', '3427', '218', '218'],
    'Finance': ['759', '76', '203', '77'],
    'Web': ['79', '883', '884','3197'],
    'CloudOps': ['187', '187']
}

# 自定义 Dumper，防止 PyYAML 自动换行列表
class InlineListDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super().increase_indent(flow=True, indentless=False)

# 生成 yaml 文件
for domain in Domain:
    datasets = DomainDataset[domain]
    for i, dataset in enumerate(datasets):
        seq = Seqlen[domain][i]
        pred_lens = [96, 192, 336] if seq == 'long' else [24, 48, 60]
        context_lens = [512] if seq == 'long' else [128]

        yaml_content = {
            "_target_": "uni2ts.data.builder.ConcatDatasetBuilder",
            "_args_": {
                "_target_": "uni2ts.data.builder.simple.generate_eval_builders",
                "dataset": f"{dataset}_eval",
                "offset": int(offset[domain][i]),
                "eval_length": int(eval_length[domain][i]),
                "prediction_lengths": pred_lens,
                "context_lengths": context_lens,
                # "patch_sizes": [32, 64] # 这里需要根据数据集在配种文件中手动设置
            }
        }

        #Each multi patch size projection is a simple Linear layer, for input projections, mapping patch size to hidden state, and for
        #output projections, mapping hidden state to distribution parameters. In practice, we pre-define the frequency to patch size
        # mapping heuristically, selecting smaller patch sizes for low frequency data and larger patch sizes for high frequency data as
        # follows:
            # • Yearly, Quarterly: 8
            # • Monthly: 8, 16, 32
            # • Weekly, Daily: 16, 32
            # • Hourly: 32, 64
            # • Minute-level: 32, 64, 128
            # • Second-level: 64, 128


        file_path = os.path.join(output_dir, f"{dataset}.yaml")
        with open(file_path, 'w') as f:
            yaml.dump(yaml_content, f, Dumper=InlineListDumper, sort_keys=False, default_flow_style=False)
        print(f"✅ Created: {file_path}")


✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/CAISO_AL.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/NYISO_AL.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/Wind.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/Covid-19.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/illness.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/weather.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/AQShunyi.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/AQWan.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/CzeLan.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/ZafNoo.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/M_DENSE.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/METR_LA.yaml
✅ Created: /home/xiongshiwei/uni2ts/cli/conf/finetune/val_data/NYCTAXI_Inflow.yaml
✅ Create